<a href="https://colab.research.google.com/github/JH-lee95/Fastspeech2-Korean/blob/master/processing_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os 
import sys

In [ ]:
!pip install jamo
!pip install konlpy
!pip install g2pk
!pip install jamotools

from jamo import h2j,hangul_to_jamo,j2hcj
from g2pk import G2p
import jamotools

In [ ]:
# 한글 문장을 초/중/종성 형태로 분리, 단독 사용 안함
g2p=G2p() #grapheme to phoneme

def jamo_split(content):

  content=g2p(content)
  jamo=h2j(content).split(" ")
  
  return jamo


In [ ]:
# word_to_phoneme 딕셔너리 , lexiocn 파일을 만드는 함수 
import tqdm

def make_p_dict(meta_path,position):

  p_dict={}

  with open(meta_path,"r") as f:
    for line in tqdm.tqdm(f.readlines()):
      line=line.rstrip()
      content=line.split("|")[position] #meta data 내의 텍스트가 기록된 위치
      word_list=content.split(" ")

      for idx,word in enumerate(word_list):
        print(word)
        if not word in p_dict.keys():
          p_dict[word]=" ".join(jamo_split(word)[0])


  # with open("phoneme_lexicon.txt","w") as p:
  #   for k,v in p_dict.items():
  #     p.write("{}\t{}\n".format(k,v))

  return p_dict
    

    

In [ ]:
p_dict=make_p_dict("/content/drive/MyDrive/korean-fastspeech2/korean_male/male/metadata.txt",1)

In [ ]:
p_dict

In [ ]:
def make_lexicon(p_dict):
  with open("p_lexicon.txt","w") as f:
    for k,v in p_dict.items():
      f.write("{}\t{}\n".format(k,v))

In [ ]:
make_lexicon(p_dict)

In [ ]:
#kss data, audio-text pair 생성 (texts are extracted from transcript.v.1.4.txt in this example)
#wav파일이 있는 곳에 같은 이름의 lab파일을 생성한다. (lab파일은 각 오디오의 발화문장을 담고 있다.)

""" 
메타데이터의 형식은 다음과 같은 형태

'폴더명/파일이름|문장'

예시)
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
1/1_0000.wav|그는 괜찮은 척하려고 애쓰는 것 같았다.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
"""

meta_name="transcript.v.1.4.txt"
base="/content/drive/MyDrive/Colab Notebooks/"


def audio_text_pair(data_dir,metadata_name):

  with open(meta_path,"r") as m:
    for line in m:
      wav_path=line.split("|")[0] #wav 파일의 위치
      content=line.split("|")[2] #문장

      text_path=wav_path.replace("wav","lab")

      with open(os.path.join(base,text_path),"w") as t:
        t.write(content)

In [ ]:
#메타데이터 만들기, wav_path : wav파일의 위치, txt_path : transcript파일의 위치, file_name : 저장하고자 하는 메타데이터의 파일명(str)
# wav-transcript는 1대1 대응하여야 함

def make_metadata(wav_path,txt_path,file_name):

  meta_txt=open(file_name,"w")

  for txt in os.listdir(txt_path):
    with open(os.path.join(txt_path,txt),"r",encoding="utf-8-sig") as f:


      content=f.readline()
      file_name=txt.replace("txt","wav")

      meta_txt.write("wav" + "/" + file_name + "|" + content)
        
  meta_txt.close()

In [ ]:
# 파일 확장자 변경


from glob import glob
#base : root_dir, before : target extension, after : expected extension , (optional)sub_dirs : list of sub_dir 
def rename_extension(base,before,after,sub_dirs=None):

  before_ext="*.{}".format(before)
  after_ext="*.{}".format(after)

  if sub_dirs:
    for dir in sub_dirs:
      dir=str(dir)
      file_path=os.path.join(base,dir)

      file_names = glob(os.path.join(file_path,before_ext))

      for name in file_names:
        src=name
        dst=src.replace(before,after)

        os.rename(src, dst)

  else:
    file_names=glob(os.path.join(base,before_ext))

    for name in file_names:
      src=name
      dst=src.replace(before,after)

      os.rename(src, dst)


In [ ]:
# 파일 이름 변경
# target : str that you want to change, expected : post-change str


import os

# base="./textgrid"
def rename_file(base,target:str,expected:str,sub_dirs=None)

  files=os.listdir(base)

  for f in files:

      before=os.path.join(base,f)
      after=os.path.join(base,f).replace(target,after)
      dst=os.rename(before,after)